In [3]:
import pandas as pd
import openpyxl
import csv
import os
import gc # Garbage Collector (Tukang bersih-bersih RAM)
import matplotlib.pyplot as plt
import seaborn as sns

# Konfigurasi Nama File
excel_filename = 'BBM AAB.xlsx'       # File Asli Anda
temp_csv_filename = 'temp_bbm_data.csv' # File Sementara (Buffer)

print("Libraries loaded")

Libraries loaded


In [4]:
def process_excel_to_disk(excel_file, output_csv):
    print(f"MEMULAI KONVERSI KE CSV (BUFFERING)")
    
    # Buka File CSV untuk ditulis
    with open(output_csv, mode='w', newline='') as f:
        writer = csv.writer(f)
        # Tulis Header CSV
        writer.writerow(['Date', 'Month', 'Unit', 'Metric', 'Value'])
        
        try:
            # Buka Excel Mode Read-Only (Ringan)
            wb = openpyxl.load_workbook(excel_file, read_only=True, data_only=True)
            total_rows = 0
            
            for sheet_name in wb.sheetnames:
                print(f"Processing Sheet: {sheet_name}...", end=" ", flush=True)
                ws = wb[sheet_name]
                row_iter = ws.iter_rows(values_only=True)
                
                try:
                    # Ambil Header (Baris 1 & 3)
                    header_1 = next(row_iter) # Unit
                    next(row_iter)            # Skip
                    header_3 = next(row_iter) # Metrik
                except StopIteration:
                    print("Kosong.")
                    continue
                
                # Mapping Kolom Valid
                valid_map = []
                current_unit = None
                for idx, (u, m) in enumerate(zip(header_1, header_3)):
                    if u: current_unit = str(u).strip()
                    if current_unit and m:
                        metric = str(m).strip().upper()
                        if metric in ['HM', 'KM', 'LITER', 'KELUAR']:
                            valid_map.append((idx, current_unit, metric))
                
                if not valid_map:
                    print("No Valid Columns.")
                    continue
                
                # Streaming Data
                count = 0
                empty_streak = 0
                for row in row_iter:
                    date_val = row[0]
                    if date_val is None:
                        empty_streak += 1
                        if empty_streak > 10: break
                        continue
                    else: empty_streak = 0
                    
                    for col_idx, unit, metric in valid_map:
                        val = row[col_idx]
                        if val is not None and val != 0:
                            writer.writerow([date_val, sheet_name, unit, metric, val])
                            count += 1
                            total_rows += 1
                            
                print(f"OK (+{count} data)")
                
                # Bersihkan Memori per Sheet (PENTING!)
                del ws
                del row_iter
                gc.collect() 
                
            wb.close()
            print(f"\n{total_rows} baris data tersimpan di '{output_csv}'.")
            return True
            
        except Exception as e:
            print(f"\nERROR: {e}")
            return False

# Jalankan Fungsi
status_konversi = process_excel_to_disk(excel_filename, temp_csv_filename)

MEMULAI KONVERSI KE CSV (BUFFERING)
Processing Sheet: JAN... OK (+10265 data)
Processing Sheet: FEB... OK (+9378 data)
Processing Sheet: MAR... OK (+10136 data)
Processing Sheet: APR... OK (+9687 data)
Processing Sheet: MEI... OK (+10406 data)
Processing Sheet: JUN... OK (+10089 data)
Processing Sheet: JUL... OK (+10562 data)
Processing Sheet: AGT... OK (+10433 data)
Processing Sheet: SEP... OK (+10192 data)
Processing Sheet: OKT... OK (+10548 data)
Processing Sheet: NOV... OK (+10087 data)

111783 baris data tersimpan di 'temp_bbm_data.csv'.


In [5]:
if status_konversi:
    print("Membaca data dari CSV...")
    
    # Baca CSV
    df = pd.read_csv(temp_csv_filename)
    
    # Rapikan Format
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
    df.dropna(subset=['Date', 'Value'], inplace=True)
    
    print(f"Data Loaded: {len(df)} baris.")
    print(df.head())
else:
    print("Stop. Perbaiki error di Cell 2 dulu.")

Membaca data dari CSV...
Data Loaded: 40675 baris.
        Date Month                     Unit  Metric    Value
0 2025-01-01   JAN      TANGKI SOLAR DEPO 4  KELUAR  2237.69
1 2025-01-01   JAN  TANGKI SOLAR DP MARUNDA  KELUAR   318.00
2 2025-01-01   JAN    TANGKI SOLAR DP PRIOK  KELUAR   228.63
3 2025-01-01   JAN   TANGKI SOLAR DP BAUBAU  KELUAR   654.00
4 2025-01-01   JAN           SPBU PERTAMINA  KELUAR   135.46


In [6]:
if 'df' in locals():
    print("Memproses Pivot Table...")
    
    # Pivot
    df_tidy = df.pivot_table(
        index=['Date', 'Month', 'Unit'], 
        columns='Metric', 
        values='Value', 
        aggfunc='sum'
    ).reset_index()
    
    # Isi NaN dengan 0
    for col in ['HM', 'LITER', 'KELUAR']:
        if col not in df_tidy.columns: df_tidy[col] = 0
        else: df_tidy[col] = df_tidy[col].fillna(0)
        
    # Kategorisasi
    def get_category(name):
        name = str(name).upper()
        if any(x in name for x in ['TANGKI', 'SPBU', 'BUNKER', 'GENSET']): return 'Storage'
        elif any(x in name for x in ['KALMAR', 'LINDE', 'CRANE', 'SMV', 'KONECRANES']): return 'Alat Berat'
        elif name.startswith('L ') or name.startswith('B '): return 'Truk'
        else: return 'Lainnya'

    df_tidy['Category'] = df_tidy['Unit'].apply(get_category)
    
    # Hitung Efisiensi (LPH)
    df_tidy.sort_values(by=['Unit', 'Date'], inplace=True)
    df_tidy['Prev_HM'] = df_tidy.groupby('Unit')['HM'].shift(1)
    df_tidy['Delta_HM'] = df_tidy['HM'] - df_tidy['Prev_HM']
    
    # Filter Error (Jam kerja minus atau > 24 jam sehari)
    df_tidy.loc[(df_tidy['Delta_HM'] < 0) | (df_tidy['Delta_HM'] > 24), 'Delta_HM'] = pd.NA
    
    # Rumus LPH
    df_tidy['LPH'] = df_tidy['LITER'] / df_tidy['Delta_HM']
    
    print("Data Siap Dianalisa!")

Memproses Pivot Table...
Data Siap Dianalisa!


In [7]:
if 'df_tidy' in locals():
    # Filter Alat Berat
    alat_berat = df_tidy[df_tidy['Category'] == 'Alat Berat']
    
    # 1. Grafik Boxplot Efisiensi
    plt.figure(figsize=(12, 6))
    top_15 = alat_berat.groupby('Unit')['LITER'].sum().nlargest(15).index
    sns.boxplot(data=alat_berat[alat_berat['Unit'].isin(top_15)], x='Unit', y='LPH')
    plt.title('Efisiensi BBM (Liter/Jam)')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 60) # Zoom in biar jelas
    plt.show()
    
    # 2. Tabel Rekonsiliasi (Gudang vs Unit)
    total_out = df_tidy[df_tidy['Category'] == 'Storage'].groupby('Month')['KELUAR'].sum()
    total_in = df_tidy[df_tidy['Category'] != 'Storage'].groupby('Month')['LITER'].sum()
    
    recon = pd.DataFrame({'Keluar Gudang': total_out, 'Masuk Unit': total_in})
    bulan_urut = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV']
    recon = recon.reindex(bulan_urut)
    
    print("\nREKONSILIASI STOK")
    print(recon)
    
    recon.plot(kind='bar', figsize=(10, 5))
    plt.title("Keluar Gudang vs Masuk Unit")
    plt.show()

: 